In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pickle
from tools.features import calculate_features

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [17]:
scaler_features = pickle.load(open('./tools/scaler_features.pkl', 'rb'))
kmeans_features = pickle.load(open('./tools/kmeans_model.pkl', 'rb'))
pca_features = pickle.load(open('./tools/pca_features.pkl', 'rb'))

In [9]:
tokensUnliked=['USDT','STARKNET','VECHAIN','LIDO','ETHENA','USDC','BTC']

In [7]:
tokens = pd.read_csv('tokensCleaned.csv',sep=';')

In [12]:
tokens.drop(tokens[tokens['symbol'].isin(tokensUnliked)].index, inplace = True)

In [13]:
features = pd.DataFrame()
for token_id, group in tokens.groupby('symbol'):
    token_features = calculate_features(group)
    token_features['symbol'] = token_id
    features = pd.concat([features, pd.DataFrame(token_features, index=[0])])

In [14]:
features.set_index('symbol', inplace=True)

In [22]:
features_scaled = scaler_features.transform(features)
features_pca = pca_features.transform(features_scaled)
features_kmeans = kmeans_features.predict(features_pca)

In [23]:
features['cluster'] = features_kmeans

In [31]:
features.groupby('cluster').size()

cluster
0     2
1    24
2     6
dtype: int64

In [36]:
features_tokens = features.reset_index()
features_tokens = features_tokens[['symbol', 'cluster']]


In [40]:
features_tokens[features_tokens['cluster']==2]

,symbol,cluster
12,FANTOM,2
13,FETCH_AI,2
14,FLOKI,2
16,GALA,2
29,THETA,2
31,XRP,2


vamos realizar 1 modelo entrenandolo con la mitad de estos token y luego evaluar el modelo con los tokens restantes en su prediccion 

In [41]:
features_tokens.to_csv('tokensForModel.csv', index=False)